## Loading pretrained model

See https://github.com/ritikjain51/CaptchaExtractor-Pytorch

In [30]:
import torch
from torch import nn
from torch.nn import functional as F

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

from models.captcha_model import CaptchaModel
from datasets.dataset import Dataset

import glob
import os

device = "cpu"

In [31]:
def label_from_filepath(filepath):
    return list(filepath.split("/")[-1].split(".")[0])


image_filepaths = glob.glob(os.path.abspath(os.path.join("../datasets/Large_Captcha_Dataset", "*.png")))
# print(image_filepaths)
labels = np.array([label_from_filepath(x) for x in image_filepaths])
# print(labels)
labels_flat = labels.flatten()
# print(labels_flat)

label_encoder = LabelEncoder().fit(labels_flat)
labels_encoded = np.apply_along_axis(label_encoder.transform, 1, labels) + 1
# print(labels_encoded)

train_X, test_X, train_y, test_y, train_target, test_target = train_test_split(
    image_filepaths, labels_encoded, labels)

train_dataset = Dataset(
    train_X,
    train_y,
    resize=(75, 300)
)

test_dataset = Dataset(
    test_X,
    test_y,
    resize=(75, 300)
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    num_workers=8,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    num_workers=8
)

In [33]:
model = CaptchaModel(len(label_encoder.classes_))
model.load_state_dict(torch.load("../models/captcha_recognition_20_epochs.pt", weights_only=True))
model.to(device)
model.eval()

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options 
	(1) Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL model.CaptchaModel was not an allowed global by default. Please use `torch.serialization.add_safe_globals([CaptchaModel])` to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.